# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [3]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from collections import Counter

In [4]:
dvach = open('2ch_corpus.txt').read()
news = open('lenta.txt').read()

In [5]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [9]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [10]:
# отрезаем по 50 предложений из каждого корпуса для расчёта перплексии

sentences_dvach = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_dvach_test = sentences_dvach[:50]
sentences_dvach = sentences_dvach[50:]

sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]
sentences_news_test = sentences_news[:50]
sentences_news = sentences_news[50:]

In [11]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [12]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix
matrix_dvach = lil_matrix((len(bigrams_dvach), len(unigrams_dvach)))

In [13]:
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

In [14]:
for ngram in trigrams_dvach:
    word0, word1, word3 = ngram.split()
    word2 = word0+' '+word1
    matrix_dvach[bigram2id_dvach[word2],word2id_dvach[word3]] =  (trigrams_dvach[ngram]/bigrams_dvach[word2])

In [15]:
matrix_dvach = csr_matrix(matrix_dvach)

In [16]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)} # словарь
id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}
for ngram in trigrams_news:
    word0, word1, word3 = ngram.split()
    word2 = word0+' '+word1
    matrix_news[bigram2id_news[word2],word2id_news[word3]] =  (trigrams_news[ngram]/bigrams_news[word2])

In [17]:
matrix_news = csr_matrix(matrix_news)

In [18]:
def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
#         chosen = np.random.choice(list(range(matrix.shape[1]))) # НЕПОН
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0]) # НЕПОН
        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
        else:        
            current_idx = bigram2id [id2bigram[current_idx].split( )[1] + ' ' + id2word[chosen]]
    return ' '.join(text)

In [19]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach).replace('<end> ', '. '))

можешь попробовать потасковать . хочу двачетянку с политача чтобы она пережила хотя бы потому что у меня было овердохуя нет . прогрессирующий паралич . их много а я забыл . анон . просрала значит просрала . попробуй да и весила примерно 110 кг . значит это украина . почему моча в чайнике — моча . рабочий сам может распорядиться 75 долларами купить сырьё оплатить ренту этц . и к тому что тебе наступили на чсв и того что она будет тебя с руками сам тоже не менее за годы простоя покрылся грязью да краска облупилась и смазать надо какой-то ванильный


In [20]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

федеральная служба охраны и обороны а также в километре восточнее села шатили которое находится за пределами своего государства 
 остановившись на отметке 25,93 доллара за акцию нк лукойл опустились на 2,98 процента и остановился на первый взгляд более сложная техника считает он 
 в случае наводнения 
 приносим извинения всем пользователям в июле движение фалуньгун было признано отсутствие средств на восстановительные и эвакуационные работы потребуется как минимум две недели когда у совета федерации и дума его повторно обратиться к президенту россии члены первого американского сетевого туристического клуба 
 инуи один из лидеров корсиканских националистов шарлю сантони 
 мы можем получить


In [45]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

по его словам прежде чем выложить на сайт playboy com 
 специалисты полагают что потери боевиков составили не менее 30 автомобилей 
 по словам авдеева несмотря на массовый характер и выбираем формат приглашенных исходя из размера среднемесячной зарплаты во втором тайме все изменилось после того как многие инфицированные лечатся самостоятельно в домашних условиях 
 сейчас работа между ними зафиксировано более 230 аварий 
 упрощается также система связи которой в результате столкновения грузовой самолет и погибли еще 12 лет 
 в среду microsoft и www noch-einmal de 
 центризбирком будет отстаивать свои честь и достоинство судьи 
 автомобильное движение по садовому


In [46]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

согласно решению федерального суда швейцарии 
 последние с этим договором авторы должны согласиться с тем аналитики на биржах резко подскочили продажи последних версий антивирусного программного обеспечения аппаратуры и всего сразу 
 главным итогом голосования является то что на одной из наиболее значительных исторических событий из тех таймеров которые были приобретены из государственного русского музея лидеры двух стран в управление фсб по чеченской республике николая кошмана в понедельник взрыв большой мощности 
 следует проводить взвешенную кадровую политику и открыть границы для иностранцев сумма эта может и не серебряные призеры первенства а ставшие третьими французы сара абитболь и стефан бернарди 
 начиная


In [47]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

согласно тому же не торопятся япония приняла решение сформировать согласительную комиссию для изучения экономических отношений 
 монтани был арестован генеральный директор оао внуковские авиалинии александр красненкер и другие службы государственной безопасности грузии был осужден рижским судом на шесть лет назад владелец рукописи запрашивал за нее кого-нибудь наказать фрацузский программист с высшим образованием серж хампич взломал систему платежных карточек 
 аушев заявил журналистам риа новости 
 иосиро мори 
 подозрительный предмет напоминающий взрывное устройство привел в действие находившийся в автомашине офицер получил смертельное ранение 
 поезд вышедший в 11 15 по 22 июня 1999 года они были обнаружены в населенных пунктах


Считаем перплексию:

In [21]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N)

In [42]:
def mean_perpl(test_set):
    perpl_mean = {'dvach':[], 'news':[]}
    perpl_sum_dvach = []
    perpl_sum_news = []
    for sent in test_set:
        prob = {'dvach':[], 'news':[]}
        for ngram in ngrammer(sent,n=3):
            word0, word1, word3 = ngram.split()
            word2 = word0+' '+word1
            if word2 in bigrams_dvach and ngram in trigrams_dvach:
                prob['dvach'].append(np.log(trigrams_dvach[ngram]/bigrams_dvach[word2]))
            else:
                prob['dvach'].append(np.log(0.00001))
            if word2 in bigrams_news and ngram in trigrams_news:
                prob['news'].append(np.log(trigrams_news[ngram]/bigrams_news[word2]))
            else:
                prob['news'].append(np.log(0.00001))
        if perplexity(prob['dvach'])!= np.inf: 
            perpl_sum_dvach.append(perplexity(prob['dvach']))
        if perplexity(prob['news'])!= np.inf: 
            perpl_sum_news.append(perplexity(prob['news']))
    perpl_mean['dvach'] = sum(perpl_sum_dvach)/len(perpl_sum_dvach)
    perpl_mean['news'] = sum(perpl_sum_news)/len(perpl_sum_news)
    return perpl_mean

In [43]:
mean_perpl(sentences_dvach_test)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


{'dvach': 30041.809295252246, 'news': 73947.63331484623}

In [44]:
mean_perpl(sentences_news_test)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


{'dvach': 60590.053280562846, 'news': 57784.239645627255}

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2. Что такое сглаживание (smoothing)?